# BMG Omega to Prism Converter

This notebook processes BMG plate data by mapping raw ratio values to a template layout.

In [ ]:
# === CONFIGURATION ===
# Define your file paths here

RAW_FILE_PATH = "/content/Plate 3 GLP1R 3.15ng Gs 1 18, HEK293A Gs KO, NN27-31.xlsx"  # Path to BMG raw data file
TEMPLATE_FILE_PATH = "/content/output_template.xlsx"  # Path to template file
OUTPUT_FILE_PATH = None  # Set to None for auto-generated name, or specify a path

# Optional: Sheet settings (use 0 for first sheet, or sheet name as string)
RAW_SHEET = 0
TEMPLATE_SHEET = 0

In [13]:
# Import libraries
from datetime import datetime
from pathlib import Path
from typing import Any
import re

import numpy as np
import pandas as pd

# Constants
DEFAULT_WELL_ROW_IDX = 9

In [ ]:
# Helper functions


def load_dataframe(file_path: str, sheet_name: int | str = 0) -> pd.DataFrame:
    """Load DataFrame from CSV or Excel file."""
    path = Path(file_path)
    if path.suffix.lower() == ".csv":
        return pd.read_csv(file_path, header=None)
    else:
        return pd.read_excel(file_path, header=None, sheet_name=sheet_name)


def generate_output_filename(raw_file_path: str) -> str:
    """Generate output filename based on the input file."""
    raw_path = Path(raw_file_path)
    raw_basename = raw_path.stem
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:17]
    output_basename = f"{raw_basename}_processed_{timestamp}.xlsx"
    output_dir = Path.cwd() / "output"
    output_dir.mkdir(exist_ok=True)
    output_file_path = output_dir / output_basename
    print(f"[INFO] Auto-generated output filename: {output_file_path}")
    return str(output_file_path)


def parse_well_columns(
    df_raw: pd.DataFrame, well_row_idx: int = DEFAULT_WELL_ROW_IDX
) -> dict[int, str]:
    """Parse raw data to map column indices to well names."""
    well_names = df_raw.iloc[well_row_idx, :].values
    col_to_well = {}
    for c, val in enumerate(well_names):
        if isinstance(val, str) and re.match(r"^[A-H]\d{2}$", val):
            col_to_well[c] = val

    print("\n--- Raw Data Analysis ---")
    print(f"[INFO] Found {len(col_to_well)} wells in raw data (row {well_row_idx}):")
    wells_list = sorted(col_to_well.values())
    print(f"  Range: {wells_list[0]} to {wells_list[-1]}")
    print(
        f"  Wells: {', '.join(wells_list[:12])}{'...' if len(wells_list) > 12 else ''}"
    )
    return col_to_well


def extract_ratio_data(
    df_raw: pd.DataFrame, col_to_well: dict[int, str]
) -> tuple[dict[tuple[int | float, str], Any], list[int | float]]:
    """Extract ratio data from the raw DataFrame."""
    ratio_start_rows = df_raw[df_raw[0].str.contains("Ratio based on", na=False)].index
    if len(ratio_start_rows) == 0:
        raise ValueError("Could not find 'Ratio based on' data block in the raw file.")

    start_row = ratio_start_rows[0]
    ratio_data = {}
    all_times = set()

    for idx in range(start_row, len(df_raw)):
        row = df_raw.iloc[idx]
        try:
            t_val = float(row[1])
            if t_val.is_integer():
                t_val = int(t_val)
            all_times.add(t_val)
        except (ValueError, TypeError):
            continue

        for c, w in col_to_well.items():
            val = row[c]
            ratio_data[(t_val, w)] = val

    sorted_times = sorted(list(all_times))
    print(
        f"Extracted data for {len(sorted_times)} time points ({min(sorted_times)}s to {max(sorted_times)}s)."
    )
    return ratio_data, sorted_times

In [15]:
# Template parsing functions


def parse_template_structure(
    df_template: pd.DataFrame,
) -> tuple[pd.DataFrame, list[tuple[int, int, str]], list[int]]:
    """Parse template to identify well positions and header rows."""
    master_strip = df_template.iloc[:, 1:3].copy()
    well_cells = []
    header_rows = []

    print("\n--- Template Structure Analysis ---")
    for r in range(len(master_strip)):
        for c in range(2):
            val = master_strip.iloc[r, c]
            if isinstance(val, str):
                if "Time [s]" in val:
                    if r not in header_rows:
                        header_rows.append(r)
                        print(
                            f"[INFO] Found header row at template row {r}, col {c}: '{val}'"
                        )
                elif re.match(r"^[A-H]\d+$", val):
                    match = re.match(r"^([A-H])(\d+)$", val)
                    w_norm = f"{match.group(1)}{int(match.group(2)):02d}"
                    well_cells.append((r, c, w_norm))

    print(f"\n[INFO] Found {len(well_cells)} well positions in template:")
    well_by_row = {}
    for r, c, w in well_cells:
        if r not in well_by_row:
            well_by_row[r] = []
        well_by_row[r].append(f"{w}(col{c})")

    for row_idx in sorted(well_by_row.keys())[:10]:
        wells_str = ", ".join(well_by_row[row_idx])
        print(f"  Row {row_idx}: {wells_str}")
    if len(well_by_row) > 10:
        print(f"  ... and {len(well_by_row) - 10} more rows")

    return master_strip, well_cells, header_rows


def verify_well_coverage(
    well_cells: list[tuple[int, int, str]], col_to_well: dict[int, str]
) -> None:
    """Verify and report well coverage between template and raw data."""
    wells_in_template = set([w for _, _, w in well_cells])
    wells_in_raw = set(col_to_well.values())

    missing_in_template = wells_in_raw - wells_in_template
    extra_in_template = wells_in_template - wells_in_raw

    if missing_in_template:
        print(
            f"\n[WARNING] Wells in raw data but NOT in template: {sorted(missing_in_template)}"
        )
    if extra_in_template:
        print(
            f"[WARNING] Wells in template but NOT in raw data: {sorted(extra_in_template)}"
        )

    matching_wells = wells_in_template & wells_in_raw
    print(
        f"\n[INFO] Matching wells: {len(matching_wells)}/{len(wells_in_raw)} from raw data"
    )
    print(
        f"[INFO] Template coverage: {len(matching_wells)}/{len(wells_in_template)} wells will have data"
    )

In [16]:
# Output generation and verification functions


def generate_output_table(
    df_template: pd.DataFrame,
    master_strip: pd.DataFrame,
    sorted_times: list[int | float],
    header_rows: list[int],
    well_cells: list[tuple[int, int, str]],
    ratio_data: dict[tuple[int | float, str], Any],
) -> pd.DataFrame:
    """Generate the final output table by combining template with ratio data."""
    result_parts = []
    result_parts.append(df_template.iloc[:, 0:1])

    for t in sorted_times:
        current_strip = master_strip.copy()
        for r in header_rows:
            current_strip.iloc[r, 0] = f"{t} (Time [s])"
            current_strip.iloc[r, 1] = np.nan

        for r, c, w_norm in well_cells:
            val = ratio_data.get((t, w_norm))
            current_strip.iloc[r, c] = val if val is not None else np.nan

        result_parts.append(current_strip)

    return pd.concat(result_parts, axis=1)


def verify_data_integrity(
    df_final: pd.DataFrame,
    ratio_data: dict[tuple[int | float, str], Any],
    sorted_times: list[int | float],
    well_cells: list[tuple[int, int, str]],
    header_rows: list[int],
) -> None:
    """Performs stringent checks to ensure output data matches input data exactly."""
    print("\n--- Starting Comprehensive Data Verification ---")

    errors = []
    checks_passed = 0
    total_checks = 0

    expected_cols = 1 + (len(sorted_times) * 2)
    if df_final.shape[1] != expected_cols:
        errors.append(
            f"Dimension Error: Expected {expected_cols} columns, found {df_final.shape[1]}"
        )
    else:
        print(f"[PASS] Output dimensions correct: {df_final.shape}")

    print("[INFO] Verifying Time Headers...")
    for t_idx, t_val in enumerate(sorted_times):
        col_idx = 1 + (t_idx * 2)
        header_r = header_rows[0]
        cell_val = df_final.iloc[header_r, col_idx]
        expected_header = f"{t_val} (Time [s])"
        if str(cell_val) != expected_header:
            errors.append(
                f"Header Mismatch at Index {t_idx}: Expected '{expected_header}', found '{cell_val}'"
            )

    print(
        f"[INFO] Verifying {len(sorted_times) * len(well_cells)} individual data points..."
    )

    for t_idx, t_val in enumerate(sorted_times):
        col_offset = 1 + (t_idx * 2)
        for r_template, c_template, w_norm in well_cells:
            total_checks += 1
            actual_col = col_offset + c_template
            output_val = df_final.iloc[r_template, actual_col]
            input_val = ratio_data.get((t_val, w_norm))

            is_match = False
            if (pd.isna(output_val) or output_val == "") and (
                pd.isna(input_val) or input_val is None
            ):
                is_match = True
            elif isinstance(output_val, (int, float)) and isinstance(
                input_val, (int, float)
            ):
                if np.isclose(output_val, input_val, equal_nan=True):
                    is_match = True
            elif output_val == input_val:
                is_match = True

            if not is_match:
                errors.append(
                    f"Data Mismatch -> Time: {t_val}s, Well: {w_norm}. Raw: {input_val}, Output: {output_val}"
                )
            else:
                checks_passed += 1

    if errors:
        print(f"\n[FAIL] Verification Failed with {len(errors)} errors.")
        for e in errors[:10]:
            print(f" - {e}")
        if len(errors) > 10:
            print(f" ... and {len(errors) - 10} more.")
        raise ValueError("Data verification failed. Output file was NOT generated.")
    else:
        print(f"[PASS] All {checks_passed} data points verified successfully.")
        print("--- Verification Complete ---\n")

In [17]:
# Load and validate files
raw_path = Path(RAW_FILE_PATH)
template_path = Path(TEMPLATE_FILE_PATH)

if not raw_path.exists():
    raise FileNotFoundError(f"Raw file not found: {RAW_FILE_PATH}")

if not template_path.exists():
    raise FileNotFoundError(f"Template file not found: {TEMPLATE_FILE_PATH}")

print(f"Processing Raw File: {RAW_FILE_PATH}")
print(f"Using Template: {TEMPLATE_FILE_PATH}")

df_raw = load_dataframe(RAW_FILE_PATH, RAW_SHEET)
df_template = load_dataframe(TEMPLATE_FILE_PATH, TEMPLATE_SHEET)

print(f"\nRaw data shape: {df_raw.shape}")
print(f"Template shape: {df_template.shape}")

Processing Raw File: data/Plate 3 GLP1R 3.15ng Gs 1 18, HEK293A Gs KO, NN27-31.xlsx
Using Template: templates/output_template.xlsx

Raw data shape: (263, 98)
Template shape: (64, 14)


In [18]:
# Parse raw data and extract ratio values
col_to_well = parse_well_columns(df_raw)
ratio_data, sorted_times = extract_ratio_data(df_raw, col_to_well)


--- Raw Data Analysis ---
[INFO] Found 96 wells in raw data (row 9):
  Range: A01 to H12
  Wells: A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12...
Extracted data for 84 time points (0s to 5395s).


In [19]:
# Parse template structure and verify well coverage
master_strip, well_cells, header_rows = parse_template_structure(df_template)
verify_well_coverage(well_cells, col_to_well)


--- Template Structure Analysis ---
[INFO] Found header row at template row 3, col 0: '0 (Time [s])'
[INFO] Found header row at template row 19, col 0: '0 (Time [s])'
[INFO] Found header row at template row 35, col 0: '0 (Time [s])'
[INFO] Found header row at template row 51, col 0: '0 (Time [s])'

[INFO] Found 96 well positions in template:
  Row 4: A01(col0), B01(col1)
  Row 5: A02(col0), B02(col1)
  Row 6: A03(col0), B03(col1)
  Row 7: A04(col0), B04(col1)
  Row 8: A05(col0), B05(col1)
  Row 9: A06(col0), B06(col1)
  Row 10: A07(col0), B07(col1)
  Row 11: A08(col0), B08(col1)
  Row 12: A09(col0), B09(col1)
  Row 13: A10(col0), B10(col1)
  ... and 38 more rows

[INFO] Matching wells: 96/96 from raw data
[INFO] Template coverage: 96/96 wells will have data


In [20]:
# Generate output table and verify data integrity
df_final = generate_output_table(
    df_template, master_strip, sorted_times, header_rows, well_cells, ratio_data
)

verify_data_integrity(df_final, ratio_data, sorted_times, well_cells, header_rows)

print(f"Output table shape: {df_final.shape}")
df_final.head(10)


--- Starting Comprehensive Data Verification ---
[PASS] Output dimensions correct: (64, 169)
[INFO] Verifying Time Headers...
[INFO] Verifying 8064 individual data points...
[PASS] All 8064 data points verified successfully.
--- Verification Complete ---

Output table shape: (64, 169)


,0,1,2,1,2,1,2,1,2,1,...,1,2,1,2,1,2,1,2,1,2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0 (Time [s]),NaN,65 (Time [s]),NaN,130 (Time [s]),NaN,195 (Time [s]),NaN,260 (Time [s]),...,5135 (Time [s]),NaN,5200 (Time [s]),NaN,5265 (Time [s]),NaN,5330 (Time [s]),NaN,5395 (Time [s]),NaN
4,NaN,0.1269,0.1273,0.1274,0.1281,0.128,0.1285,0.1284,0.1291,0.1287,...,0.122,0.1223,0.122,0.1228,0.1215,0.121,0.1213,0.1222,0.1228,0.122
5,NaN,0.1736,0.1728,0.1749,0.1745,0.176,0.1755,0.1769,0.1761,0.1771,...,0.1483,0.1474,0.1473,0.1471,0.1471,0.1458,0.1472,0.1465,0.1475,0.1453
6,NaN,0.1714,0.1712,0.1736,0.1736,0.1753,0.175,0.176,0.1754,0.1766,...,0.1477,0.1474,0.1474,0.1478,0.1474,0.1474,0.147,0.1474,0.1469,0.1467
7,NaN,0.1663,0.1668,0.1703,0.1722,0.1723,0.1739,0.1732,0.1758,0.1742,...,0.1476,0.1483,0.145,0.1481,0.1464,0.1474,0.1474,0.1473,0.1475,0.1483
8,NaN,0.1595,0.1573,0.1652,0.1643,0.1679,0.1667,0.1698,0.1689,0.171,...,0.1474,0.1486,0.1486,0.1477,0.1485,0.1487,0.148,0.1475,0.1483,0.1472
9,NaN,0.147,0.1456,0.1533,0.1521,0.1564,0.1552,0.1586,0.1578,0.1602,...,0.1488,0.1497,0.1488,0.1489,0.1492,0.1489,0.1489,0.1483,0.1483,0.1485


In [21]:
# Save output file
output_path = (
    OUTPUT_FILE_PATH if OUTPUT_FILE_PATH else generate_output_filename(RAW_FILE_PATH)
)

df_final.to_excel(output_path, index=False, header=False)

print(
    f"\n✓ Successfully processed {len(sorted_times)} time points across {len(well_cells)} wells"
)
print(f"✓ Output saved to: {output_path}")

[INFO] Auto-generated output filename: /Users/skn506/Documents/tidy_BMG_microplate/output/Plate 3 GLP1R 3.15ng Gs 1 18, HEK293A Gs KO, NN27-31_processed_20260108_151550_9.xlsx

✓ Successfully processed 84 time points across 96 wells
✓ Output saved to: /Users/skn506/Documents/tidy_BMG_microplate/output/Plate 3 GLP1R 3.15ng Gs 1 18, HEK293A Gs KO, NN27-31_processed_20260108_151550_9.xlsx
